In [1]:
import os
%pwd

'/Users/whysocurious/Documents/MLDSAIProjects/e2e-mlops-gcp/research'

In [2]:
os.chdir("../")
%pwd

'/Users/whysocurious/Documents/MLDSAIProjects/e2e-mlops-gcp'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    color: str
    year: int
    month: int
    root_dir: Path
    source_URL: str
    local_data: Path
    local_data_file: Path

In [4]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories


In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        params = self.params.dataDetails

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            color=params.color,
            year=params.year,
            month=params.month,
            root_dir=config.root_dir,
            source_URL=config.source_URL + f'{params.color}_tripdata_{params.year:04d}-{params.month:02d}.parquet',
            local_data=config.local_data,
            local_data_file=config.local_data_file + f'{params.color}_tripdata_{params.year:04d}-{params.month:02d}.parquet'
        )

        return data_ingestion_config

In [6]:
import os
from mlProject import logger
from mlProject.utils.common import get_size
import pandas as pd

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            # filename, headers = request.urlretrieve(
            #     url = self.config.source_URL,
            #     filename = self.config.local_data_file
            # )
            filename = self.config.local_data_file
            pd.read_parquet(self.config.source_URL).to_parquet(filename)
            logger.info(f"{filename} download! \n")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    # def extract_zip_file(self):
    #     """
    #     zip_file_path: str
    #     Extracts the zip file into the data directory
    #     Function returns None
    #     """
    #     unzip_path = self.config.unzip_dir
    #     os.makedirs(unzip_path, exist_ok=True)
    #     with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
    #         zip_ref.extractall(unzip_path)
  

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    # data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-30 13:22:15,254: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-30 13:22:15,257: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-30 13:22:15,258: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-30 13:22:15,259: INFO: common: created directory at: artifacts]
[2024-06-30 13:22:15,260: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-30 13:22:41,715: INFO: 340874643: artifacts/data_ingestion/yellow_tripdata_2023-03.parquet download! 
]
